### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from neuropy import plotting
from neuropy.analyses.brainstates import correlation_emg
import subjects

### Test best channels for dual probes 

In [ ]:
from neuropy.core import Signal

sessions = subjects.sd.ratSday3

for s, sess in enumerate(tqdm(sessions)):
    signal = sess.eegfile.get_signal(t_start=15000, t_stop=20000)
    window = 0.5
    overlap = 0
    fs = 1 / (window - overlap)
    emgs = []
    for dist in [0, 30, 100, 400, 800]:
        emg = correlation_emg(
            signal=signal,
            probe=sess.probegroup,
            window=0.5,
            overlap=0,
            n_jobs=6,
            min_dist=dist,
        )
        emgs.append(emg)

    # emg_signal = Signal(traces=emg[np.newaxis, :], sampling_rate=fs, t_start=window / 2)

In [ ]:
_, ax = plt.subplots()

for i in range(5):
    ax.plot(emgs[i])

### Generate correlation emg for all sessions
- Correlation emg's are calculated using power in high frequency bands (300-600 Hz).

In [ ]:
from neuropy.core import ProbeGroup
from neuropy.core import Signal

# sessions = subjects.ripple_sess()
sessions = subjects.sd.ratKday1
# sessions = subjects.nsd.ratKday2

for s, sess in enumerate(tqdm(sessions)):
    signal = sess.eegfile.get_signal()
    window = 1
    overlap = 0
    fs = 1 / (window - overlap)
    # emg = correlation_emg(
    #     signal=signal, probe=sess.probegroup, window=window, overlap=overlap, n_jobs=8
    # )
    emg_signal = Signal(traces=emg[np.newaxis, :], sampling_rate=fs, t_start=window / 2)
    emg_signal.save(sess.filePrefix.with_suffix(".emg"))

### Interpolate or exclude extreme values in EMG

In [ ]:
from scipy.ndimage import gaussian_filter1d
from neuropy.utils.signal_process import FourierSg

sessions = subjects.nsd.ratSday2

for s, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    # emg_signal = sess.emg
    # emg = gaussian_filter1d(emg_signal.traces[0], 20)
    # indices = (emg_signal.time > post[0]) & (emg_signal.time < post[0] + 5 * 3600)
    # emg[indices] = gaussian_filter1d(emg[indices], 30)
    # emg[emg > 0.5] = 0.3
    corr_emg = gaussian_filter1d(sess.emg.traces[0], 20)
    # channels = [50, 61, 55, 29, 13, 2, 14]
    # # channels =
    # emg_power = []
    # for chan in channels:
    #     signal = sess.eegfile.get_signal(chan)
    #     spect = FourierSg(signal, window=2, overlap=1, norm_sig=True)
    #     emg_power.append(spect.freq_slice(300).traces.sum(axis=0))
    # emg_power = gaussian_filter1d(np.array(emg_power), 20, axis=-1).sum(axis=0)

In [ ]:
_, ax = plt.subplots(2, 1)

ax[0].plot(sess.emg.time, stats.zscore(sess.emg.traces[0]))
# ax[1].plot(sess.eegfile.get, sess.eegfile.get_signal(33).traces[0])

In [ ]:
indices = emg_power > 0.026
# emg_power_new = np.interp(spect.time, spect.time[~indices], emg_power[~indices])
emg_power_new = emg_power.copy()
emg_power_new[indices] = np.nan
emg_power_new = pd.DataFrame(emg_power_new).interpolate(method="linear")[0].to_numpy()

In [ ]:
emg_power2 = emg_power_new * np.interp(spect.time, sess.emg.time, corr_emg)

In [ ]:
from neuropy.core import Signal

emg_signal = Signal(traces=emg_power_new[np.newaxis, :], sampling_rate=1, t_start=2 / 2)
emg_signal.save(sess.filePrefix.with_suffix(".emg"))

In [ ]:
_, ax = plt.subplots()
# emg_power = spect.freq_slice(300).traces.sum(axis=0)
# ax.plot(emg)
bins = np.linspace(0.014, 0.03, 100)
hist_emg = np.histogram(emg_power, bins)[0]
# ax.plot(bins[:-1], hist_emg)
ax.plot(emg_power2)
# ax.set_ylim(0, top=0.03)
# ax.set_yscale("log")

### Compare distributions of EMG values

In [ ]:
from scipy.ndimage import gaussian_filter1d
from neuropy.analyses.brainstates import hmmfit1d

sessions = subjects.ripple_sess()
# sessions = subjects.sd.ratSday3
bins = np.linspace(-1, 5, 500)

emg_info = {}
for s, sess in enumerate(sessions):
    emg = sess.emg
    smth_emg = gaussian_filter1d(emg.traces[0], sigma=10 / 0.5)
    zsc_emg = stats.zscore(smth_emg, nan_policy="omit")
    means_prior = np.array([-0.1, 0.2])[:, np.newaxis]
    transmat = np.array([[0.8, 0.2], [0.2, 0.8]])
    startprob = np.array([0.1, 0.9])
    # labels, mus = hmmfit1d(
    #     zsc_emg,
    #     ret_means=True,
    #     means_prior=means_prior,
    #     transmat_prior=transmat,
    #     startprob_prior=startprob,
    # )
    hist_emg = np.histogram(zsc_emg, bins)[0]
    emg_info[sess.animal.name + sess.animal.day] = dict(
        emg=zsc_emg,
        # means=mus,
        # labels=labels,
        dist=hist_emg,
    )

# zsc_emg = np.concatenate(zsc_emg)

In [ ]:
_, axs = plt.subplots(len(emg_info) + 1, 2, width_ratios=[4, 1])
# axs = axs.reshape(-1)

for i, name in enumerate(emg_info):
    emg_dict = emg_info[name]
    ax_emg = axs[i, 0]
    ax_emg.plot(emg_dict["emg"], zorder=1)
    ax_emg.set_ylabel(name)
    ax_label = ax_emg.twinx()
    # ax_label.plot(emg_dict["labels"], "r", zorder=0)

    ax_hist = axs[i, 1]
    ax_hist.plot(bins[:-1], emg_dict["dist"])
    # ax_hist.axvline(emg_dict["means"][0])
    # ax_hist.axvline(emg_dict["means"][1])

    # ax_hist.set_yscale("log")

### Best separation for high and low EMG

In [ ]:
from sklearn.cluster import KMeans
from scipy.ndimage import gaussian_filter1d
from neuropy.analyses.brainstates import hmmfit1d
from neuropy.utils.signal_process import FourierSg

# sessions = subjects.ripple_sess()
sess = subjects.nsd.ratNday2[0]

emg = sess.emg
time = emg.time
emg = gaussian_filter1d(emg.traces[0], sigma=10 / 0.5)
noisy_epochs = sess.brainstates["NOISE"]

noisy_bool = np.zeros_like(time).astype("bool")
if noisy_epochs is not None:
    noisy_arr = noisy_epochs.as_array()
    for noisy_ind in range(noisy_arr.shape[0]):
        st = noisy_arr[noisy_ind, 0]
        en = noisy_arr[noisy_ind, 1]
        noisy_bool[np.where((time >= st) & (time <= en))[0]] = True

emg = emg[~noisy_bool]
time = time[~noisy_bool]
emg_log = np.log10(emg)

In [ ]:
from neuropy.analyses.brainstates import gaussian_classify
from math_utils import schmidt_trigger_threshold

emg_labels, emg_fit_params = gaussian_classify(emg_log, ret_params=True)
emg_hist, emg_bins = np.histogram(emg_log, 200, density=True)
emg_means = emg_fit_params["means"].squeeze()
emg_covs = emg_fit_params["covariances"].squeeze()
emg_weights = emg_fit_params["weights"]
lowfit = (
    stats.norm.pdf(emg_bins, float(emg_means[0]), np.sqrt(float(emg_covs[0])))
    * emg_weights[0],
)

highfit = (
    stats.norm.pdf(emg_bins, float(emg_means[1]), np.sqrt(float(emg_covs[1])))
    * emg_weights[1],
)
full_fit = lowfit[0] + highfit[0]

In [ ]:
from math_utils import schmidt_trigger_threshold

bins_between_peaks = (emg_bins > emg_means[0]) & (emg_bins < emg_means[1])
thresh_ind = np.argmin(full_fit[bins_between_peaks])
thresh_val = emg_bins[bins_between_peaks][thresh_ind]
low_thresh = (emg_means[0] + thresh_val) / 2
high_thresh = (emg_means[1] + thresh_val) / 2

over_high = emg_log >= high_thresh
below_low = emg_log <= low_thresh

emg_labels2 = schmidt_trigger_threshold(emg_log, low_thresh, high_thresh)

In [ ]:
_, ax = plt.subplots()

ax.plot(emg_bins[:-1], emg_hist)
# ax.plot(emg_bins[:-1],lowfit[0])
# ax.plot(emg_bins[:-1],highfit[0])
# ax.plot(emg_bins[:-1], lowfit[0] + highfit[0])
ax.axvline(thresh_val)
ax.axvline(low_thresh)
ax.axvline(high_thresh)

In [ ]:
_, ax = plt.subplots(3, 1, sharex=True)

ax[0].plot(emg_log)
ax[0].axhline(high_thresh)
ax[0].axhline(thresh_val)
ax[0].axhline(low_thresh)
ax[1].plot(emg_labels)
ax[2].plot(emg_labels2)

In [ ]:
theta_log = np.log10(theta_spect)[emg_labels.astype("bool")]
theta_labels, theta_means = gaussian_fit1d(theta_log, ret_means=True)
theta_bins, theta_hist = get_hist(theta_spect)


_, axs = plt.subplots(2, 2, width_ratios=[4, 1], sharex="col")
axs[0, 0].plot(theta_log)
# axs[1, 0].plot(labels)
axs[1, 0].plot(theta_labels)
# axs[1, 0].plot(labels_new)
# axs[1, 0].plot(clus.score_samples(feat))

# axs[0, 1].plot(bins[:-1], hist_emg)
axs[1, 1].plot(theta_bins[:-1], theta_hist)
# axs[1, 1].axvline(theta_means[0])
# axs[1, 1].axvline(theta_means[1])

In [ ]:
from sklearn.mixture import GaussianMixture

emg_log = np.log10(smth_emg)
max_bound = emg_log.mean() + 1 * emg_log.std()
min_bound = emg_log.mean() - 1 * emg_log.std()

good_indx = (emg_log > min_bound) & (emg_log < max_bound)
feat = emg_log[:, None]
feat_bins = np.linspace(feat[:, 0].min(), feat[:, 0].max(), 500)
feat_hist = np.histogram(feat[:, 0], feat_bins)[0]
init = np.array([-0.8, -0.6])[:, None]
# kmeans = KMeans(n_clusters=2, n_init=1, init=init).fit(zsc_emg[:, None])
clus = GaussianMixture(n_components=2).fit(feat[good_indx, :])
# labels = kmeans.labels_
clus_means = clus.means_

_, axs = plt.subplots(2, 2, width_ratios=[4, 1], sharex="col")
axs[0, 0].plot(feat[:, 0])
# axs[1, 0].plot(labels)
axs[1, 0].plot(clus.predict(feat))
# axs[1, 0].plot(clus.score_samples(feat))

# axs[0, 1].plot(bins[:-1], hist_emg)
axs[1, 1].plot(feat_bins[:-1], feat_hist)
axs[1, 1].axvline(clus_means[0])
axs[1, 1].axvline(clus_means[1])